In [1]:
import psana
from psana import *

In [2]:
def retrieve_cspad_data(evt):
    """
    Retrieve intensities collected on CsPad from input event
    
    ...
    """
    data = list()
    cspad = evt.get(psana.CsPad.DataV2, psana.Source('DscCsPad'))
    for num in range(cspad.quads_shape()[0]):
        data.append(cspad.quads(num).data())
    return data

In [3]:
geom_file="/cds/data/psdm/cxi/cxic0415/calib/CsPad::CalibV1/CxiDs1.0:Cspad.0/geometry/0-end.data"

In [4]:
#start_time = time.time()

ds = psana.DataSource('exp=cxic0415:run=36:smd')
det = Detector(geom_file) 

for num,evt in enumerate(ds.events()):
    if num==100: break
print(evt.keys()[1])

data = retrieve_cspad_data(evt)
image = det.assemble_image_stack(data)

#print(f"elapsed time is {(time.time() - start_time)/60.0} min")

KeyError: 'Missing Detector: Source string not found in configStore: /cds/data/psdm/cxi/cxic0415/calib/CsPad::CalibV1/CxiDs1.0:Cspad.0/geometry/0-end.data'

In [16]:
plt.imshow(image)
plt.axis('off')

NameError: name 'plt' is not defined